# Import Libraries

In [3]:

from PIL import Image
import numpy as np
import os
import cv2
import keras
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

Using TensorFlow backend.


In [6]:


import numpy as np 
import pandas as pd 

import os
print(os.listdir(r"C:\Users\91701\Downloads\cell-images-for-detecting-malaria\cell_images"))

# Any results you write to the current directory are saved as output.

['Parasitized', 'Uninfected']


# Data Prepearaion

In [8]:
data=[]
labels=[]
Parasitized=os.listdir(r"C:/Users/91701/Downloads/cell_images/cell_images/Parasitized/")
for a in Parasitized:
    try:
        image=cv2.imread(r"C:/Users/91701/Downloads/cell_images/cell_images/Parasitized/"+a)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(0)
    except AttributeError:
        print("")

Uninfected=os.listdir(r"C:/Users/91701/Downloads/cell_images/cell_images/Uninfected/")
for b in Uninfected:
    try:
        image=cv2.imread(r"C:/Users/91701/Downloads/cell_images/cell_images/Uninfected/"+b)
        image_from_array = Image.fromarray(image, 'RGB')
        size_image = image_from_array.resize((50, 50))
        data.append(np.array(size_image))
        labels.append(1)
    except AttributeError:
        print("")
    

In [9]:
Cells=np.array(data)
labels=np.array(labels)

In [10]:
np.save("Cells",Cells)
np.save("labels",labels)

In [11]:
Cells=np.load("Cells.npy")
labels=np.load("labels.npy")

In [12]:
s=np.arange(Cells.shape[0])
np.random.shuffle(s)
Cells=Cells[s]
labels=labels[s]

In [13]:
num_classes=len(np.unique(labels))
len_data=len(Cells)

## train-test split 

In [14]:
(x_train,x_test)=Cells[(int)(0.1*len_data):],Cells[:(int)(0.1*len_data)]
x_train = x_train.astype('float32')/255 
x_test = x_test.astype('float32')/255
train_len=len(x_train)
test_len=len(x_test)

In [15]:
(y_train,y_test)=labels[(int)(0.1*len_data):],labels[:(int)(0.1*len_data)]

# One hot encoding:

In [16]:

y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

# Create Sequential Model:

In [17]:
#creating sequential model
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=2,padding="same",activation="relu",input_shape=(50,50,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=32,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=64,kernel_size=2,padding="same",activation="relu"))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(2,activation="softmax"))#2 represent output layer neurons 
model.summary()






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 50, 50, 16)        208       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 64)        8256      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
______________________

In [18]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:

model.fit(x_train,y_train,batch_size=50,epochs=20,verbose=1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/20





24803/24803 [==============================] - 69s 3ms/step - loss: 0.3489 - acc: 0.8387
Epoch 2/20
24803/24803 [==============================] - 64s 3ms/step - loss: 0.1533 - acc: 0.9519
Epoch 3/20
24803/24803 [==============================] - 63s 3ms/step - loss: 0.1349 - acc: 0.9567
Epoch 4/20
24803/24803 [==============================] - 64s 3ms/step - loss: 0.1229 - acc: 0.9598
Epoch 5/20
24803/24803 [==============================] - 64s 3ms/step - loss: 0.1151 - acc: 0.9612
Epoch 6/20
24803/24803 [==============================] - 66s 3ms/step - loss: 0.1079 - acc: 0.9628
Epoch 7/20
24803/24803 [==============================] - 65s 3ms/step - loss: 0.1019 - acc: 0.9648
Epoch 8/20
24803/24803 [==============================] - 64s 3ms/step - loss: 0.0978 - acc: 0.9663
Epoch 9/20
24803/24803 [==============================] - 65s 3ms/step - loss: 0.0914 - acc: 0.967

# Checking accuracy

In [20]:
accuracy = model.evaluate(x_test, y_test, verbose=1)
print('\n', 'Test_Accuracy:-', accuracy[1])

2755/2755 [==============================] - 3s 931us/step

 Test_Accuracy:- 0.9542649727767695


# Saving the model weights

In [22]:
from keras.models import load_model
model.save('cells.h5')